# Double-check tweet language with TextCat

Source: http://www.let.rug.nl/~vannoord/TextCat/

In [1]:
import os
import pandas as pd
import pipes
import re
import tempfile
from nltk.classify import textcat
from IPython.display import clear_output

In [2]:
DATADIR = "/home/erikt/projects/puregome/data/text/"
DATADIRTEXTCAT = "/home/erikt/projects/puregome/data/textcat/"
IDSTR = "id_str"
TEXT = "text"
TEXTCATDIR = "/home/erikt/software/textcat/"
TEXTCAT = "text_cat"
PERL = "/usr/bin/perl"
TEXTUNKNOWN = "I don't know; Perhaps this is a language I haven't seen before?"
UNKNOWN = "unknown"
OR = " or "
DUTCH = "dutch"
LANG = "lang"
OTHER = "other"
DUTCHTEXTCAT = "nld"

In [3]:
def squeal(text=None):
    clear_output(wait=True)
    if not text is None: print(text)

In [4]:
def createPipe(command):
    pipe  = pipes.Template()
    pipe.append(command,"--")
    return(pipe)

def writeAllPipeInput(pipe,text):
    tmpFile = tempfile.NamedTemporaryFile(delete=False)
    f = pipe.open(tmpFile.name,"w")
    f.write(text)
    f.close()
    return(tmpFile.name)

def readPipeOutput(tmpFileName):
    tmpFile = open(tmpFileName,"r")
    textOut = ""
    for line in tmpFile: textOut += line
    tmpFile.close()
    os.unlink(tmpFileName)
    return(textOut)

def textcatString2List(text):
    if text == TEXTUNKNOWN: return([UNKNOWN])
    else: return(text.split(OR))

def textcatLocal(textIn):
    pipe = createPipe(PERL+" "+TEXTCATDIR+TEXTCAT)
    tmpFileName = writeAllPipeInput(pipe,textIn)
    textOut = readPipeOutput(tmpFileName)
    return(textcatString2List(textOut.strip()))

In [11]:
FILEPATTERN = "202007..-00|202007..-12"
MAXTWEETS = 1000

tc = textcat.TextCat()
inFiles = sorted(os.listdir(DATADIR))
outFiles = sorted(os.listdir(DATADIRTEXTCAT))
for inFileName in inFiles:
    if re.search(FILEPATTERN,inFileName): # and not inFileName in outFiles:
        df = pd.read_csv(DATADIR+inFileName,index_col=IDSTR)[:MAXTWEETS]
        lang = []
        for i in range(0,len(df)):
            if (i+1) % 100 == 0: squeal(str(i+1)+" "+str(len(df))+" "+inFileName)
            text = re.sub(r"\\n"," ",df.iloc[i][TEXT])
            textcatOut = textcatLocal(text.lower())
            if DUTCH in textcatOut: lang.append(DUTCH)
            elif TEXTUNKNOWN in textcatOut: lang.append(UNKNOWN)
            else: lang.append(textcatOut[0])
        df[LANG] = lang
        df.to_csv(DATADIRTEXTCAT+inFileName,compression="gzip")

1000 1000 20200713-12.out.gz
